# Graphillionを使いこなすために

この章ではGraphillionを利用してグラフに関する問題を効率的に解くための実践的なノウハウを紹介します．

Graphillionを用いた計算の効率はグラフ集合を表すZDDの大きさによって決まります．`GraphSet`オブジェクトを操作する途中で巨大なZDDが作られてしまうと，その部分が処理のボトルネックとなる可能性があります．


残念ながら，あるグラフ集合を表すZDDがどの程度大きくなるかを事前に予測することは難しいことが知られています．一方で，どういう場面でZDDが大きくなるかについてはいくつかの経験則があります．

この章ではZDDの大きさに関する経験則と，ZDDが大きくなったときにとれる対策を説明します．


## ZDDの性質

### グラフ集合の要素数とZDDサイズは比例しない

グラフ集合の大きさとZDDの大きさは必ずしも比例しません．

In [ ]:
from graphillion import GraphSet, tutorial
from tutorial_util import zdd_size, draw_zdd
import networkx as nx

In [ ]:
GraphSet.set_universe(tutorial.grid(4, 4))
paths = GraphSet.paths(1, 25)
all_graphs = GraphSet({})

len(paths), zdd_size(paths), len(all_graphs), zdd_size(all_graphs)

(8512, 605, 1099511627776, 40)

上のコードでは，全ての部分グラフの集合を表す`all_graphs`と，グリッドグラフの対角頂点間の経路の集合を表す`paths`とを作成して，対応するZDDサイズを比較しています．グラフ集合の要素数は`all_graphs`のほうが圧倒的に多いですが，ZDDサイズを比較すると`all_graphs`に対応するZDDのほうが小さくなっていることが分かります．



###  `universe`が大きくなるとZDDは指数的に大きくなる

ZDDを用いると，組合せ爆発を起こすグラフ集合を圧縮して小さく表現することができます．しかし，圧縮後が効くとはいえ，`universe`が大きくなるとZDDサイズも指数的に増加する傾向があります．

In [ ]:
zdd_sizes = []
num_paths = []
for i in range(2, 8):
    GraphSet.set_universe(tutorial.grid(i, i))
    paths = GraphSet.paths(1, (i+1)**2)
    num_paths.append(len(paths))
    zdd_sizes.append(zdd_size(paths))
    
zdd_sizes, num_paths

経験的には，`universe`に指定したグラフの頂点が数百頂点以上になってくると，Graphillionの処理が現実的な時間で終わらなくなります．

### 密なグラフだとZDDが大きくなる傾向がある．

`universe`に設定したグラフが密な場合，つまり頂点の数に対して辺の数が多い場合，ZDDサイズが増大する傾向があります．

In [ ]:
grid = tutorial.grid(3, 3)
complete = nx.complete_graph(7)

complete = [(i+1, j+1) for (i, j) in complete.edges()]
len(grid), len(complete) # グラフの辺の数

(24, 21)

辺の数を揃えたグリッドグラフと完全グラフとで，ZDDの大きさがどう変わるかを見てみましょう．

まずはグリッドグラフの例です．

In [ ]:
grid_zdd_sizes = {}

GraphSet.set_universe(grid)

grid_zdd_sizes['cycle'] = zdd_size(GraphSet.cycles())
grid_zdd_sizes['tree'] = zdd_size(GraphSet.trees(1, is_spanning=True ))
grid_zdd_sizes

{'cycle': 109, 'tree': 189}

次に完全グラフの例です．

In [ ]:
grid_zdd_sizes = {}

GraphSet.set_universe(complete)

grid_zdd_sizes['cycle'] = zdd_size(GraphSet.cycles())
grid_zdd_sizes['tree'] = zdd_size(GraphSet.trees(1, is_spanning=True ))
grid_zdd_sizes

{'cycle': 479, 'tree': 1087}

密なグラフである完全グラフを扱った場合に，ZDDがより大きくなることが確認できました．

### ランダムなグラフ集合を避ける

グラフ集合に含まれる部分グラフに何らかの規則性があると，ZDDによる圧縮が効果的に機能します．一方で，部分グラフがランダムに選択されるような場合，巨大なZDDがつくられる傾向があります．

In [ ]:
GraphSet.set_universe(tutorial.grid(5, 5))

all_graphs = GraphSet({})

random_graphs = GraphSet([]) # 空のグラフ集合として初期化

for i, g in enumerate(all_graphs.rand_iter()):
    if i == 1000:
        break
    random_graphs = random_graphs.union(GraphSet([g])) # ランダムに部分グラフを追加
zdd_size(random_graphs)    

20978

上の例では，全ての部分グラフの集合を表す`all_graphs`から，ランダムに部分グラフを1000個取り出して`random_graphs`を作成しています．1000個の部分グラフの集合である`random_graphs`を表現するのに，20000頂点以上のZDDを構築する必要があるので，ZDDの圧縮が効いていないといえます．

## ZDDの肥大化を避けるために

巨大なZDDが構築されると，計算に長い時間がかかったり，あるいは計算機のメモリを消費し尽くす可能性があります．そのような事態を回避するために取れる対応策をいくつか紹介します．

### メモリを潤沢に搭載した計算機を利用する

Graphillionを利用するうえでボトルネックとなりやすいのがメモリです．構築されるZDDの大きさによっては，Graphillionは簡単に数GB単位のメモリを消費します．もし大きなグラフを対象とした処理を検討しているのなら，メモリを潤沢に搭載した計算機を利用することを推奨します．

可能なら10GB以上のメモリを搭載した計算機が利用できるとできることが広がります．

### 演算の順序を工夫する

Graphillionでは複数の`GrpahSet`オブジェクトを合成したりフィルタリングしたりすることによって所望の`GraphSet`を構築します．その途中で巨大なZDDが構築されると，それがボトルネックとなり計算の効率が低下します．このボトルネックは演算の順番を工夫することで回避できる可能性があります．

いま，グリッドグラフの対角頂点を結ぶ経路のうち，辺の数が12未満のものを求めたいとします．

まずは経路の集合を求めてから辺の数の制約を加える方法で`GraphSet`を構築したときに，途中で構築されるZDDの大きさがどう変化するか見ていきましょう．

In [ ]:
GraphSet.set_universe(tutorial.grid(5, 5))

zdd_sizes = []
gs = GraphSet.paths(1, 36)
zdd_sizes.append(zdd_size(gs))

gs = gs.smaller(12)
zdd_sizes.append(zdd_size(gs))

zdd_sizes

[5635, 106]

次に，まず辺の数が12未満である`GraphSet`を構築して，そこから対角頂点間の経路となっているものを取りだす方法で`GraphSet`オブジェクトを作ってみましょう．

In [ ]:
zdd_sizes = []
gs = GraphSet({}) # 全ての部分グラフの集合をつくる．
zdd_sizes.append(zdd_size(gs))

gs = gs.smaller(12) # 長さ12未満の部分グラフの集合をつくる
zdd_sizes.append(zdd_size(gs))

gs = gs.paths(1, 36)
zdd_sizes.append(zdd_size(gs))

zdd_sizes

[60, 550, 106]

ここで，`gs.paths(1, 36)`はグラフ集合`gs`から頂点1, 36間の経路となっているものを取り出した`GraphSet`オブジェクトを作成するメソッドです．ほかにも`gs.cycles`, `gs.trees`などのように，特定のグラフ集合に特化した構築法を`gs`のメソッドとして利用することができます．

構築順序を変えることで，途中で作られるZDDのサイズを抑えることができました．

### 変数順序を調整する

ZDDの頂点のラベルには順序があります．以下のZDDだと，根からどのように枝をたどっても，頂点のラベルは1, 2, 3の順番で出現するようになっています．この，ラベルの出現順序のことを**変数順序**とよびます．ZDDを構築する際は，まず変数順序を決めてから，その変数順序に従うようなZDDを構築します．

<img src="https://github.com/nsnmsak/graphillion_tutorial/blob/master/ja/img/09/sample_zdd.png?raw=1" alt="ZDDの例" style="height: 300px;"/>

変数順序が変わるとZDDの大きさが（場合によっては指数的に）変化します．GraphillionでもZDDの変数順序を適切に設定することで効率的な処理を実現できます．

Graphillionでは，`GraphSet.set_universe()`で初期化をするときにあわせて変数順序のパラメータを指定できます．では，変数順序を変化させると何が起きるかを見てみましょう．

In [ ]:
grid = tutorial.grid(4,4)
GraphSet.set_universe(grid, traversal='as-is')  
paths = GraphSet.paths(1, 25)
zdd_size(paths)

577

`GraphSet.set_universe`の引数`traversal`では変数順序を決める方法を指定します．引数が省略された場合，Graphillionは`universe`のグラフを探索することで適当な変数順序を設定します．`traversal='as-is'`とすると，与えられたグラフの辺の順序をそのまま利用します．上の例では`grid`の辺の順序をそのまま採用しています．

次に`grid`の辺の順序を変化させたときに何が起きるかみてみましょう．

In [ ]:
from random import sample

grid_shuffled = sample(grid, len(grid))
print(grid_shuffled)
GraphSet.set_universe(grid_shuffled, traversal='as-is')
paths = GraphSet.paths(1, 25)
zdd_size(paths)

[(7, 8), (21, 22), (6, 7), (9, 14), (3, 8), (18, 19), (16, 21), (17, 22), (14, 15), (14, 19), (3, 4), (2, 7), (9, 10), (1, 2), (8, 9), (8, 13), (11, 16), (15, 20), (5, 10), (24, 25), (20, 25), (11, 12), (19, 20), (4, 5), (13, 14), (16, 17), (12, 17), (23, 24), (12, 13), (19, 24), (17, 18), (10, 15), (7, 12), (6, 11), (18, 23), (22, 23), (1, 6), (4, 9), (2, 3), (13, 18)]


33464

`sample`はリストをランダムに並べ替えるメソッドです．`grid`の変数順序をランダムに並べ替えると，ZDDサイズが急増することが分かります．

`universe`に設定したグラフに応じたよい変数順序を与えることでZDDサイズを小さくすることができます．しかし，ZDDを小さくするような変数順序を求める問題自体も**NP困難**であることが知られているため，よい変数順序が常に求まるわけではありません．

よい変数順序を求める方法はいくつか知られていますが，ここでは変数順序探索のパラメータを調整する方法を紹介します．Grapphillionでは`GraphSet.set_universe()`メソッドによる初期化時に変数順序の探索処理を実行しています．このメソッドの引数`source`に設定する頂点を変えることで，異なる変数順序を得ることができます．

In [ ]:
grid = tutorial.grid(4, 6)

zdd_sizes = []
for i in range(5):
    GraphSet.set_universe(grid, source=(5 * i + 1))
    paths = GraphSet.paths(1, 35)
    zdd_sizes.append(zdd_size(paths))
zdd_sizes

[6215, 4408, 6703, 4709, 6861]

引数`source`はどの頂点を起点として変数順序の探索をはじめるかを定めます．`source`に指定ｓる頂点の値を変化させることでZDDの大きさが変化することが分かります．

## この章のまとめ

この章ではGraphillionで効率的な処理を行うために知っておくべきZDDの挙動を紹介しました．